In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import skorch
from skorch import NeuralNetClassifier

from pathlib import Path
import os
import sys
import time
import copy

import pandas as pd
import matplotlib.pylab as plt

import numpy as np
from numpy import dstack

# Local modules
from cub_tools.train import train_model
from cub_tools.visualize import imshow, visualize_model
from cub_tools.utils import unpickle, save_pickle
from cub_tools.transforms import makeDefaultTransforms
from cub_tools.ensembles import stackedEnsemble

In [2]:
# Script runtime options
model_names = ['resnet152', 'resnext101_32x8d', 'inception_v3', 'googlenet']
data_parallel = {'resnet152' : False,
                 'inceptionv4' : True,
                 'resnext101_64x4d' : True, 
                 'pnasnet5large' : True, 
                 'googlenet' : False,
                 'inception_v3' : False, 
                 'resnext101_32x8d' : False}
data_root_dir = '../data'
model_root_dir = '../models'
stages = ['train', 'test']


# Paths setup
data_dir = os.path.join(data_root_dir,'images')

# Ensemble setup
run_train_data_stack = False
run_test_data_stack = False
ensemble_path = '../models/classification/ensemble_1'

In [3]:
os.makedirs(ensemble_path,exist_ok=True)

In [4]:
# Get data transforms
data_transforms = makeDefaultTransforms()

In [5]:
# Setup data loaders with augmentation transforms
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in stages}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in stages}
dataset_sizes = {x: len(image_datasets[x]) for x in stages}
class_names = image_datasets[stages[0]].classes

In [6]:
# Setup the device to run the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device::', device)

Device:: cuda:0


In [7]:
models = {}

for model_name in model_names:
    
    print('[INFO] Loading model {}'.format(model_name))
    
    # Paths
    output_dir = os.path.join(model_root_dir,'classification/{}'.format(model_name))
    model_file = os.path.join(output_dir, 'caltech_birds_{}_full.pth'.format(model_name))
    
    # Load the best model from file
    models[model_name] = torch.load(model_file)
    if data_parallel[model_name]:
        models[model_name] = torch.nn.DataParallel(model[model_name])
    models[model_name] = models[model_name].to(device)

[INFO] Loading model resnet152
[INFO] Loading model resnext101_32x8d
[INFO] Loading model inception_v3
[INFO] Loading model googlenet


# Ensemble Methods Part 1 - Stacking Torchvision models

Different approaches for ensembling, first lets try stacking.

Stacking ensemble bespoke implementation for PyTorch

Following the tutorial here: https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/

## Creating the stacked datasets for train and test sets

This process pushes the training and test images through the CNN models trained on the image classification problem, and stacks their predicted class probabilities into a single array. Assuming there N images in the set, and M models in the ensemble, the expected size of the array is as follows:

nx (cols): number of classes by number of models e.g. 4 models and 200 classes, 800 columns.
ny (rows): number of images

The array is arrange as follows:

    image 1: |---200 class probability columns model 1---|---200 class probability columns model 2---|---***---|---200 class probability columns model M---|
    image 2: |---200 class probability columns model 1---|---200 class probability columns model 2---|---***---|---200 class probability columns model M---|
    ***
    ***
    image N: |---200 class probability columns model 1---|---200 class probability columns model 2---|---***---|---200 class probability columns model M---| 

## Fit the ensemble of models

The stacking ensemble takes either the set of PyTorch trained models and the PyTorch dataloader object and runs the dataset stacking prior to fitting the meta learner of the ensemble, or it takes the pre-stacked dataset of class predictions from the image set and simply fits the classifier object.

The classifier object as default is a Logistic Regression. Any scikit-learn type classifier object can be passed to the fitting function, as long as it has a **fit** and **predict** method. Optional hyperparameter arguments can be provided to initialise the classifier object, and these can be passed in a dictionary with the *meta_learner_options* variable. 

In [10]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRFClassifier

### Default Logistic Regression

Run predictions using the test set stacked dataset and run classification metrics to assess the model performance

In [9]:
mtlearner1 = stackedEnsemble(models=models, device=device)

In [10]:
mtlearner1.fit(dataloader=dataloaders['train'])
mtlearner1.save_stacked_dataset(fname_path=ensemble_path)

[INFO] Creating the meta learner inputs (probabilities from individual models) as none provided.
[INFO] Starting StackX..0..5..10..15..20..25..30..35..40..45..50..55..60..65..70..75..80..85..90..95..100..105..110..115..120..125..130..135..140..145..150..155..160..165..170..175..180..185..190..195..200..205..210..215..220..225..230..235..240..245..250..255..260..265..270..275..280..285..290..295..300..305..310..315..320..325..330..335..340..345..350..355..360..365..370..Complete
[INFO] Training the meta learner.....Complete
[IO] Saved stacked training set input data to: ../models/classification/ensemble_1/train_stackX.pkl
[IO] Saved stacked training set labels data to: ../models/classification/ensemble_1/train_stacky.pkl


In [11]:
y_pred, y_true = mtlearner1.predict(dataloader=dataloaders['test'], return_ytrue=True)
mtlearner1.save_stacked_dataset(dstack_type='test', fname_path=ensemble_path)

[INFO] Creating the meta learner inputs (probabilities from individual models) as none provided.
[INFO] Starting StackX..0..5..10..15..20..25..30..35..40..45..50..55..60..65..70..75..80..85..90..95..100..105..110..115..120..125..130..135..140..145..150..155..160..165..170..175..180..185..190..195..200..205..210..215..220..225..230..235..240..245..250..255..260..265..270..275..280..285..290..295..300..305..310..315..320..325..330..335..340..345..350..355..360..Complete
[INFO] Predicting with the meta learner.....Complete
[IO] Saved stacked test set input data to: ../models/classification/ensemble_1/test_stackX.pkl
[IO] Saved stacked test set labels data to: ../models/classification/ensemble_1/test_stacky.pkl


In [12]:
mtlearner1.class_report(y_true=y_true)

              precision    recall  f1-score   support

           0       0.84      0.87      0.85        30
           1       0.96      0.87      0.91        30
           2       0.84      0.96      0.90        28
           3       0.91      0.97      0.94        30
           4       0.87      0.93      0.90        14
           5       1.00      0.91      0.95        11
           6       1.00      0.96      0.98        23
           7       0.81      0.94      0.87        18
           8       0.56      0.48      0.52        29
           9       0.97      0.93      0.95        30
          10       0.63      0.63      0.63        30
          11       0.93      0.96      0.94        26
          12       0.97      0.97      0.97        30
          13       0.94      0.97      0.95        30
          14       0.93      0.93      0.93        28
          15       1.00      0.93      0.96        28
          16       1.00      0.93      0.96        27
          17       1.00    

In [14]:
mtlearner1.save(fname_path=ensemble_path, fname_pre='LogisticRegression')

### Test ability to used pre-stacked datasets to train the meta learner

In [8]:
stackX = unpickle(os.path.join(ensemble_path,'train_stackX.pkl'))
stacky = unpickle(os.path.join(ensemble_path,'train_stacky.pkl'))
test_stackX = unpickle(os.path.join(ensemble_path,'test_stackX.pkl'))
test_stacky = unpickle(os.path.join(ensemble_path,'test_stacky.pkl'))

In [19]:
mtlearner1.fit_stacked(stackX=stackX, stacky=stacky)

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Training the meta learner.....Complete


In [20]:
y_pred = mtlearner1.predict_stacked(stackX=test_stackX)

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Predicting with the meta learner...

In [21]:
mtlearner1.class_report(y_true=test_stacky, y_pred=y_pred)

              precision    recall  f1-score   support

           0       0.84      0.87      0.85        30
           1       0.96      0.87      0.91        30
           2       0.84      0.96      0.90        28
           3       0.91      0.97      0.94        30
           4       0.87      0.93      0.90        14
           5       1.00      0.91      0.95        11
           6       1.00      0.96      0.98        23
           7       0.81      0.94      0.87        18
           8       0.56      0.48      0.52        29
           9       0.97      0.93      0.95        30
          10       0.63      0.63      0.63        30
          11       0.93      0.96      0.94        26
          12       0.97      0.97      0.97        30
          13       0.94      0.97      0.95        30
          14       0.93      0.93      0.93        28
          15       1.00      0.93      0.96        28
          16       1.00      0.93      0.96        27
          17       1.00    

## SKLearn Random Forests

In [24]:
mtlearner2_1 = stackedEnsemble(models=models, device=device,
                             meta_learner=RandomForestClassifier,
                             meta_learner_options={'n_estimators' : 250, 'verbose' : 10, 'n_jobs' : -1})
mtlearner2_1.fit_stacked(stackX=stackX, stacky=stacky)
y_pred = mtlearner2_1.predict_stacked(stackX=test_stackX)
mtlearner2_1.class_report(y_true=test_stacky, y_pred=y_pred)
mtlearner2_1.save(fname_path=ensemble_path, fname_pre='RandomForest250')

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Training the meta learner...building tree 1 of 250
building tree 2 of 250
building tree 3 of 250
building tree 4 of 250
building tree 5 of 250
building tree 6 of 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.


building tree 7 of 250
building tree 8 of 250
building tree 9 of 250
building tree 10 of 250
building tree 11 of 250
building tree 12 of 250


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.4s


building tree 13 of 250
building tree 14 of 250
building tree 15 of 250
building tree 16 of 250
building tree 17 of 250
building tree 18 of 250


...

building tree 240 of 250
building tree 241 of 250
building tree 242 of 250
building tree 243 of 250
building tree 244 of 250
building tree 245 of 250
building tree 246 of 250
building tree 247 of 250
building tree 248 of 250
building tree 249 of 250
building tree 250 of 250


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.6min finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    0.2s


..Complete
[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Predicting with the meta learner...

[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 209 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 230 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 250 out of 250 | elapsed:    0.6s finished


              precision    recall  f1-score   support

           0       0.84      0.87      0.85        30
           1       0.96      0.87      0.91        30
           2       0.87      0.96      0.92        28
           3       0.97      0.97      0.97        30
           4       0.87      0.93      0.90        14
           5       0.91      0.91      0.91        11
           6       0.95      0.91      0.93        23
           7       0.77      0.94      0.85        18
           8       0.54      0.45      0.49        29
           9       0.97      0.97      0.97        30
          10       0.73      0.63      0.68        30
          11       0.93      1.00      0.96        26
          12       0.97      0.97      0.97        30
          13       0.93      0.93      0.93        30
          14       0.93      0.96      0.95        28
          15       0.93      0.96      0.95        28
          16       0.93      0.96      0.95        27
          17       0.94    

In [25]:
mtlearner2_2 = stackedEnsemble(models=models, device=device,
                             meta_learner=RandomForestClassifier,
                             meta_learner_options={'n_estimators' : 500, 'verbose' : 10, 'n_jobs' : -1})
mtlearner2_2.fit_stacked(stackX=stackX, stacky=stacky)
y_pred = mtlearner2_2.predict_stacked(stackX=test_stackX)
mtlearner2_2.class_report(y_true=test_stacky, y_pred=y_pred)
mtlearner2_2.save(fname_path=ensemble_path, fname_pre='RandomForest500')

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Training the meta learner...building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.


building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s



building tree 12 of 500


...

building tree 475 of 500
building tree 476 of 500
building tree 477 of 500
building tree 478 of 500
building tree 479 of 500
building tree 480 of 500
building tree 481 of 500
building tree 482 of 500
building tree 483 of 500
building tree 484 of 500
building tree 485 of 500
building tree 486 of 500
building tree 487 of 500
building tree 488 of 500
building tree 489 of 500
building tree 490 of 500
building tree 491 of 500
building tree 492 of 500
building tree 493 of 500
building tree 494 of 500
building tree 495 of 500
building tree 496 of 500
building tree 497 of 500
building tree 498 of 500
building tree 499 of 500
building tree 500 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.2min finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    0.1s


..Complete
[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Predicting with the meta learner...

[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 209 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 230 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 253 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 301 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 326 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Do

              precision    recall  f1-score   support

           0       0.84      0.87      0.85        30
           1       0.96      0.87      0.91        30
           2       0.87      0.96      0.92        28
           3       0.97      0.97      0.97        30
           4       0.87      0.93      0.90        14
           5       0.91      0.91      0.91        11
           6       0.95      0.91      0.93        23
           7       0.77      0.94      0.85        18
           8       0.52      0.45      0.48        29
           9       0.97      0.97      0.97        30
          10       0.73      0.63      0.68        30
          11       0.93      1.00      0.96        26
          12       0.97      0.97      0.97        30
          13       0.93      0.93      0.93        30
          14       0.93      0.96      0.95        28
          15       0.96      0.96      0.96        28
          16       0.93      0.96      0.95        27
          17       0.94    

## XGBoost Classifiers

Look at Random Forest and Gradient Boosting Machine classifiers

In [11]:
mtlearner3_1 = stackedEnsemble(meta_learner=XGBClassifier,
                             meta_learner_options={'n_estimators' : 250, 
                                                   'verbosity' : 1, 
                                                   'n_jobs' : 6, 
                                                   'objective' : 'multi:softmax', 
                                                   'num_class' : 200,
                                                   'tree_method' : 'gpu_hist'})
mtlearner3_1.fit_stacked(stackX=stackX, stacky=stacky)
y_pred = mtlearner3_1.predict_stacked(stackX=test_stackX)
mtlearner3_1.class_report(y_true=test_stacky, y_pred=y_pred)
mtlearner3_1.save(fname_path=ensemble_path, fname_pre='GBM250')

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Training the meta learner.....Complete
[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Predicting with the meta learner...              precision    recall  f1-score   support

           0       0.79      0.73      0.76        30
           1       0.86      0.80      0.83        30
           2       0.81      0.89      0.85        28
           3       1.00      0.87      0.93        30
           4       0.76      0.93      0.84        14
           5       0.89      0.73      0.80        11
           6       0.86      0.83      0.84        23
           7       0.80      0.89      0.84        18
           8       0.50      0.52      0.51        29
           9       1.00      0.90      0.95        30
          10       0.67      0.53      0.59        30
          11       0.96      1.00      0.98        26
          12       0.94      0.97      0.95      

In [12]:
mtlearner3_2 = stackedEnsemble(meta_learner=XGBClassifier,
                             meta_learner_options={'n_estimators' : 500, 
                                                   'verbosity' : 1, 
                                                   'n_jobs' : 6, 
                                                   'objective' : 'multi:softmax', 
                                                   'num_class' : 200,
                                                   'tree_method' : 'gpu_hist'})
mtlearner3_2.fit_stacked(stackX=stackX, stacky=stacky)
y_pred = mtlearner3_2.predict_stacked(stackX=test_stackX)
mtlearner3_2.class_report(y_true=test_stacky, y_pred=y_pred)
mtlearner3_2.save(fname_path=ensemble_path, fname_pre='GBM500')

[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Training the meta learner.....Complete
[INFO] Stacked input table and labels found, using these to train meta learner.
[INFO] Predicting with the meta learner...              precision    recall  f1-score   support

           0       0.79      0.73      0.76        30
           1       0.86      0.80      0.83        30
           2       0.81      0.89      0.85        28
           3       1.00      0.87      0.93        30
           4       0.76      0.93      0.84        14
           5       0.89      0.73      0.80        11
           6       0.86      0.83      0.84        23
           7       0.80      0.89      0.84        18
           8       0.50      0.52      0.51        29
           9       1.00      0.90      0.95        30
          10       0.67      0.53      0.59        30
          11       0.96      1.00      0.98        26
          12       0.94      0.97      0.95      